## The main aim of this notebook is to collect sale informtaion for comic books

[Comichron](http://www.comichron.com/) is a website collecting book sale data for comic books, for a given book the following is given:

__Month-Year__, __Comic-book Title__, __Issue__, __Price__, __Publisher__, __Est. sales__

In [1]:
%pylab inline
import pandas as pd
import pickle as pk

Populating the interactive namespace from numpy and matplotlib


In [2]:
from urllib import urlopen
from bs4 import BeautifulSoup

In [3]:
def get_page_soup(url, month, year):
    import requests
    from requests.auth import HTTPProxyAuth
    
    txheaders = {   
    'Accept' : 'application/xml',
    'User-agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36',
    'Accept-Language': 'en-us',
    'Accept-Encoding': 'gzip, deflate, compress;q=0.9',
    'Keep-Alive': '600',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Accept-Encoding': 'identity',
    'server': 'Apache',
    'vary': 'Accept-Encoding',
    'Referer': 'http://dl.acm.org/'
    }

    
    api = 'xyz' # <--- this needs to be filled in with authentication info
    proxy_host = 'proxy.crawlera.com'
    proxy_auth = HTTPProxyAuth(api, '')
    proxies = {"http": "http://{}:8010/".format(proxy_host)}

    # ready for https
    if url.startswith("https:"):
        url = "http://" + url[8:]
        headers["X-Crawlera-Use-HTTPS"] = "1"
    if month<10:
        month = '0'+str(month)
    new_url = url %(str(year), str(year), str(month))
    print new_url
    # scrape
    stream = False
    r = requests.get(new_url, headers=txheaders, proxies=proxies, auth=proxy_auth, stream=stream)

    bsoup = BeautifulSoup(r.content, 'html.parser')
    return bsoup

In [4]:
def get_title_ls(bsoup):
    tags = bsoup.findAll('td', {'class':"xl73"})
    if len(tags)<100:
        tags = bsoup.findAll('td', {'class':"xl74"})
    if len(tags)<100:
        tags = bsoup.findAll('td', {'class':"xl72"})
    _ls = []

    for t in tags:
        _ls += [t.text.encode('ascii', 'ignore')]
    return _ls

def get_issue_ls(bsoup):
    tags = bsoup.findAll('td', {'class':"xl70"})
    if len(tags)<100:
        tags = bsoup.findAll('td', {'class':"xl69"})
    _ls = []

    for t in tags:
        _ls += [t.text.encode('ascii', 'ignore')]
    return _ls

def get_price_ls(bsoup):
    tags = bsoup.findAll('td', {'class':"xl79"})
    if len(tags)<100:
        tags = bsoup.findAll('td', {'class':"xl81"})
    if len(tags)<100:
        tags = bsoup.findAll('td', {'class':"xl80"})
    _ls = []

    for t in tags:
        _ls += [t.text.encode('ascii', 'ignore')]
    return _ls

def get_puber_ls(bsoup):
    tags = bsoup.findAll('td', {'class':"xl66"})
    _ls = []

    for t in tags:
        _ls += [t.text.encode('ascii', 'ignore')]
    return _ls

def get_sales_ls(bsoup):
    tags = bsoup.findAll('td', {'class':"xl67"})
    _ls = []

    for t in tags:
        _ls += [t.text.encode('ascii', 'ignore')]
    return _ls

In [5]:
def get_page_info_tuples(url, m, y):
    
    bsoup = get_page_soup(url, m, y)
    titles = get_title_ls(bsoup) 
    issues = get_issue_ls(bsoup)
    prices = get_price_ls(bsoup)
    publisher = get_puber_ls(bsoup)
    sales = get_sales_ls(bsoup)
    zeros = np.zeros(len(sales))

    return zip(zeros, titles, issues, prices, publisher, sales)

In [6]:
def get_page_info_df(url, m, y):
    page_tups = get_page_info_tuples(url, m, y)
    print len(page_tups)

    cols = ['Month-Year', 'Comic-book Title', 'Issue', 'Price', 'Publisher', 'Est. sales']
    df = pd.DataFrame(page_tups, columns=cols)
    return df

In [7]:
url = 'http://www.comichron.com/monthlycomicssales/%s/%s-%s.html'

In [8]:
soup = get_page_soup(url, 1, 2010)

http://www.comichron.com/monthlycomicssales/2010/2010-01.html



## Start scraping


In [9]:
df2 = get_page_info_df(url, 1, 2001)

http://www.comichron.com/monthlycomicssales/2001/2001-01.html
321


In [10]:
df2.head(2)

,Month-Year,Comic-book Title,Issue,Price,Publisher,Est. sales
0,0.0,X-Men,390,$2.25,Marvel,"100,964"
1,0.0,Uncanny X-Men,2,$2.25,Marvel,"98,437"


In [11]:
import time

url = 'http://www.comichron.com/monthlycomicssales/%s/%s-%s.html'
cols = ['Month-Year', 'Comic-book Title', 'Issue', 'Price', 'Publisher', 'Est. sales']

df_my = {}
df = pd.DataFrame(columns=cols)
for y in range(2004,2012):
    time.sleep(10)
    for m in range(1,13):
       
        time.sleep(10)
        if m <10:
            m = '0'+str(m) 
        
        my = str(y)+'-'+str(m)+'-01'
        print my
        
        tmp_df = get_page_info_df(url, m, y)
        tmp_df.loc[:,'Month-Year'] = pd.Series(len(tmp_df)*[my], index= tmp_df.index, dtype=str)
        l = len(tmp_df)
        if l<100:
            print y, m, l
        df = df.append(tmp_df, ignore_index=True)
        
        

        df_my[my] = df


2004-01-01
http://www.comichron.com/monthlycomicssales/2004/2004-01.html
341
2004-02-01
http://www.comichron.com/monthlycomicssales/2004/2004-02.html
394
2004-03-01
http://www.comichron.com/monthlycomicssales/2004/2004-03.html
391
2004-04-01
http://www.comichron.com/monthlycomicssales/2004/2004-04.html
390
2004-05-01
http://www.comichron.com/monthlycomicssales/2004/2004-05.html
392
2004-06-01
http://www.comichron.com/monthlycomicssales/2004/2004-06.html
396
2004-07-01
http://www.comichron.com/monthlycomicssales/2004/2004-07.html
393
2004-08-01
http://www.comichron.com/monthlycomicssales/2004/2004-08.html
385
2004-09-01
http://www.comichron.com/monthlycomicssales/2004/2004-09.html
386
2004-10-01
http://www.comichron.com/monthlycomicssales/2004/2004-10.html
389
2004-11-01
http://www.comichron.com/monthlycomicssales/2004/2004-11.html
392
2004-12-01
http://www.comichron.com/monthlycomicssales/2004/2004-12.html
385
2005-01-01
http://www.comichron.com/monthlycomicssales/2005/2005-01.html
393

In [12]:
def get_tags_given_class_ls(bsoup, class_ls):
    tags = bsoup.findAll('td', {'class':"xl73"})
    for cls in class_ls:
        
        if len(tags)<100:
            tags = bsoup.findAll('td', {'class':"xl74"})
    if len(tags)<100:
        tags = bsoup.findAll('td', {'class':"xl72"})
    _ls = []

    for t in tags:
        _ls += [t.text.encode('ascii', 'ignore')]
    return _ls


In [13]:
for y in range(2012,2013):
    time.sleep(10)
    for m in range(1,13):
       
        time.sleep(10)

        tmp_df = get_page_info_df(url, m, y)
        tmp_df.loc[:,'Month-Year'] = pd.Series(len(tmp_df)*[my], index= tmp_df.index, dtype=str)
        l = len(tmp_df)
        if l<100:
            print y, m, l
        df = df.append(tmp_df, ignore_index=True)
        if m <10:
            m = '0'+str(m) 
        
        my = str(y)+'-'+str(m)+'-01'
        print my
        
        

        df_my[my] = df


http://www.comichron.com/monthlycomicssales/2012/2012-01.html
329
2012-01-01
http://www.comichron.com/monthlycomicssales/2012/2012-02.html
328
2012-02-01
http://www.comichron.com/monthlycomicssales/2012/2012-03.html
1
2012 3 1
2012-03-01
http://www.comichron.com/monthlycomicssales/2012/2012-04.html
326
2012-04-01
http://www.comichron.com/monthlycomicssales/2012/2012-05.html
330
2012-05-01
http://www.comichron.com/monthlycomicssales/2012/2012-06.html
331
2012-06-01
http://www.comichron.com/monthlycomicssales/2012/2012-07.html
1
2012 7 1
2012-07-01
http://www.comichron.com/monthlycomicssales/2012/2012-08.html
325
2012-08-01
http://www.comichron.com/monthlycomicssales/2012/2012-09.html
1
2012 9 1
2012-09-01
http://www.comichron.com/monthlycomicssales/2012/2012-10.html
1
2012 10 1
2012-10-01
http://www.comichron.com/monthlycomicssales/2012/2012-11.html
0
2012 11 0
2012-11-01
http://www.comichron.com/monthlycomicssales/2012/2012-12.html
0
2012 12 0
2012-12-01


In [14]:
cols = ['Month-Year', 'Comic-book Title', 'Issue', 'Price', 'Publisher', 'Est. sales']

sales_df = pd.DataFrame(columns=cols)
for y in df_my:
    if int(y[:3])<2012:
        sales_df = sales_df.append(df_my[y], ignore_index=True)

In [24]:
sales_df.head(2)

,Month-Year,Comic-book Title,Issue,Price,Publisher,Est. sales
0,2004-01-01,Ultimate Fantastic Four,151,1,Marvel,"126,729"
1,2004-01-01,New X-Men,152,2,Marvel,"124,048"


In [21]:
com_dir = '/Users/admin/Dropbox/Comic_Book_Sales/Datasets/'
sales_df.to_csv(com_dir+'sales_df.csv', sep=',', encoding='utf-8')
pk.dump(sales_df, open(com_dir+'comic_book_sales.pk', 'wb'), protocol=2)